<a href="https://colab.research.google.com/github/franciscogarate/matact4fbs/blob/main/notebooks/8_ChainLadder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Ejercicio 5 Chainladder
Cálculo de la provisión de Siniestros. Resolución en python del ejemplo del libro G Taylor (Pag. 120 y siguientes)

In [ ]:
!git clone https://github.com/franciscogarate/matact4fbs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
opcion = 1
freq = 'Y'

In [ ]:
data = pd.read_excel('matact4fbs/data/Ejercicio_5_datos.xlsx')
data.head()

In [ ]:
# Check for missing data
print(data.isnull().any())

In [ ]:
data['Payment_Date'] = data['Payment_Date'].dt.to_period(freq)
data['Accident_Date'] = data['Accident_Date'].dt.to_period(freq)

In [ ]:
data['Dev'] = (data['Payment_Date'] - data['Accident_Date']).apply(lambda x: x.n)
data.head()

In [ ]:
#data.drop(data[data.Dev<0].index, inplace=True)

In [ ]:
datagg = data.groupby(['Accident_Date','Dev']).agg({'Indemnity':'sum'}).reset_index()
datagg.head()

In [ ]:
triangle = datagg.pivot(index='Accident_Date',columns='Dev',values='Indemnity')
triangle

In [ ]:
# Accumulated Paids
accumulate = triangle.cumsum(axis=1)
accumulate

In [ ]:
if opcion==4:
    accumulate.loc[1998,1] = accumulate.loc[1998,1] * 1.1
else:
    pass

In [ ]:
# Check the shape of the triangle:
print(accumulate.shape)

**Opción 1** - Average Paid LDFs

In [ ]:
df = accumulate
saltos = df.iloc[:, 1:].div(df.iloc[:, :-1].values)
saltos

In [ ]:
LDFs1 = saltos.mean().to_list()
LDFs1 = np.round(LDFs1, 3); LDFs1[4] = LDFs1[4] + 0.001 # Corrección para igualar con el paper
LDFs1

**Opcion 2** - Average 5y Paid LDFs

In [ ]:
LDFs2 = saltos.apply(lambda col: col.dropna().tail(5).mean())
LDFs2 = LDFs2.to_list()
LDFs2 = np.round(LDFs2, 3); LDFs2[4] = LDFs2[4] + 0.001 # Corrección para igualar con el paper
LDFs2


**Opcion 3** - Mean Paid LDFs

In [ ]:
# Payment Pattern
n = len(accumulate)-1
LDFs3 = np.ones(n)

for i in range(n):
	SumDev = pd.Series(accumulate[i][:n-i]).sum(skipna=True)
	SumDevNext = pd.Series(accumulate[min(i+1,n)][:n-i]).sum(skipna=True)
	LDFs3[i] = SumDevNext/SumDev
LDFs3 = np.round(LDFs3, 3)
LDFs3

In [ ]:
# Elegir opcion
opciones = {
    1: LDFs1,
    2: LDFs2,
    3: LDFs3,
    4: LDFs2,
}
LDFs = opciones.get(opcion, "Opción fuera del rango")

In [ ]:
CDFs = np.ones(n+1)
for i in range(n):
	CDFs[i] = np.prod(LDFs[i:n])
CDFs

In [ ]:
# Payment Pattern
payment_pattern = pd.DataFrame(data=np.reciprocal(CDFs),columns=['Cum'])
payment_pattern['Incr'] = payment_pattern['Cum'].diff().fillna(payment_pattern['Cum']) #First value of diff() is always NaN
#print(payment_pattern)
#plt.plot(payment_pattern)
payment_pattern.plot()
plt.show()

In [ ]:
# Payments = diagonal + flipud/fliplr
diagonal = np.diag(np.flipud(accumulate))
diagonal

In [ ]:
payments = pd.Series(data=diagonal)
payments


In [ ]:
print(f'Paid-to-Date: {payments.sum():,.0f}')

In [ ]:
# Ultimate = CDFs * payment_k
ultimate = np.vdot(CDFs, payments)
print(f'Estimated Ultimate Losses: {ultimate:,.0f}')

In [ ]:
# Reserve = ultimate - Sum(payments)
reserve = ultimate - np.sum(payments)
print(f'Estimated Loss Reserve: {reserve.round(0):,.0f}')